***

<div class="alert alert-block alert-warning">
<b>API KEYS:</b> Zen Rows y ScrapperAPI fueron de paga para permitir un scrapper más fluido
</div>

### Using ScrapperApi

In [1]:
import requests
import time

def safe_request(url):
    """Keep trying the request until it succeeds."""
    while True:
        try:
            payload = { 'api_key': 'cef04c9c0f3b5be06796e360914d4955', 'url': str(url) }
            response = requests.get('https://api.scraperapi.com/', params=payload)
            print("Respuesta de servidor: ", response)
            if response.status_code == 200:
                return response
            else:
                print(f"Respuesta no exitosa, código de estado: {response.status_code}. Reintentando...")
        except (ConnectionError, ChunkedEncodingError) as e:
            print(f"Error de conexión: {e}. Reintentando...")
            time.sleep(300)  # Espera 30 segundos antes de reintentar



In [2]:
import requests
import time

def safe_request_detail(url):
    """Keep trying the request until it succeeds."""
    while True:
        try:
            payload = { 'api_key': 'cef04c9c0f3b5be06796e360914d4955', 'url': str(url) }
            response = requests.get('https://api.scraperapi.com/', params=payload)
            if response.status_code == 200:
                return response
            else:
                print(f"Respuesta no exitosa, código de estado: {response.status_code}. Reintentando...")
        except (ConnectionError, ChunkedEncodingError) as e:
            print(f"Error de conexión: {e}. Reintentando...")
            time.sleep(300)  # Espera 30 segundos antes de reintentar

### Putting it all together

### ALL CODE

In [3]:
import csv
from datetime import datetime
from bs4 import BeautifulSoup

def get_url(position, location):
    """Generate url from position and location"""
    template = 'https://www.indeed.com/jobs?q={}&l={}'
    position = position.replace(' ', '+')
    location = location.replace(' ', '+')
    url = template.format(position, location)
    return url

def get_record(card, occ):
    job_title = card.h2.a.span.get('title') if card.h2 and card.h2.a and card.h2.a.span else 'No title'
    company = card.find('span', {"data-testid":"company-name"}).text.strip() if card.find('span', {"data-testid":"company-name"}) else 'No company'
    job_location = card.find('div', {"data-testid":"text-location"}).text.strip() if card.find('div', {"data-testid":"text-location"}) else 'No location'
    post_date = card.find('span', {"data-testid":"myJobsStateDate"}).text.strip() if card.find('span', {"data-testid":"myJobsStateDate"}) else 'No date provided'
    today = datetime.today().strftime('%Y-%m-%d')
    summary = card.find('div', 'css-9446fg eu4oa1w0').text.strip().replace('\n', ' ') if card.find('div', 'css-9446fg eu4oa1w0') else 'No summary'
    job_url = 'https://www.indeed.com' + card.h2.a.get('href')

    response_Detail = safe_request_detail(job_url)
    soupDetail = BeautifulSoup(response_Detail.text, 'html.parser')

    salary_tag = soupDetail.find('div', 'js-match-insights-provider-tvvxwd')
    salary = salary_tag.text.strip() if salary_tag else ''

    details = soupDetail.find('div', {"id":"jobDescriptionText"}).text.strip().replace('\n', ' ') if soupDetail.find('div', {"id":"jobDescriptionText"}) else "No details provided"
    
    record = (occ, job_title, company, job_location, post_date, today, summary, details, salary, job_url)
    return record


def main(occ, location):
    url = get_url(occ, location)
    page_count = 0  # Inicializar el contador de páginas

    with open('indeed_scrapper/indeed_scrapper_June_SanFrancisco.csv', 'a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f, quoting=csv.QUOTE_ALL, escapechar='\\')
        # Check if the file is empty to write headers
        if f.tell() == 0:
            writer.writerow(['Occ', 'JobTitle', 'Company', 'Location', 'PostDate', 'ExtractDate', 'Summary', 'Details', 'Salary', 'JobUrl'])

        while page_count < 7:  # Límite de 7 páginas
            response = safe_request(url)
            soup = BeautifulSoup(response.text, 'html.parser')
            cards = soup.find_all('div', 'job_seen_beacon')
            print("Número de secciones encontradas:", len(cards))
            if not cards:
                print("No se encontraron trabajos")
                break

            for card in cards:
                record = get_record(card, occ)
                writer.writerow(record)

            next_page = soup.find('a', {'data-testid': 'pagination-page-next'})
            if next_page:
                url = 'https://wwww.indeed.com' + next_page.get('href')
                page_count += 1
                print("Siguiente url: ", url)
            else:
                print("Se llego a la ultima página")
                break


In [4]:
occupations_list = [
    "ICT sales professional",
    "Marketing manager",
    "Product analyst",
    "Product manager",
    "Product designer",
    "Business Intelligence professional",
    "Infraestructure engineer",
    "Computer Systems Analyst",
    "Software infrastructure architect",
    "Web developer",
    "Software developer",
    "App developer",
    "User interface designer",
    "Software engineer",
    "Software architect",
    "Software Quality Assurance Analysts and Testers",
    "Embedded systems engineer",
    "Web and Digital Interface Designers",
    "Database infrastructure engineer",
    "Network architect",
    "Database Administrators",
    "Database Architects",
    "Network and Computer Systems Administrator",
    "Artificial intelligence engineer",
    "Machine Learning engineer",
    "Data science engineer",
    "Data analyst",
    "Data scientist",
    "Artificial Intelligence scientist",
    "Data architect",
    "ICT security specialist",
    "IT security operations",
    "Information Security Analysts",
    "Product Security and IT Security Integration Specialist",
    "Product risk specialist",
    "Security architect",
    "Database support engineer",
    "Data center operations engineer",
    "Support systems engineer",
    "Computer Network Support Specialists"
]


In [5]:
for occ in occupations_list:
    print(occ)
    main(occ, 'Mexico')

# run the main program

ICT sales professional
Respuesta de servidor:  <Response [200]>
Número de secciones encontradas: 1
Se llego a la ultima página
Marketing manager
Respuesta de servidor:  <Response [200]>
Número de secciones encontradas: 15
Siguiente url:  https://mx.indeed.com/jobs?q=Marketing+manager&l=Mexico&start=10
Respuesta de servidor:  <Response [200]>
Número de secciones encontradas: 15
Siguiente url:  https://mx.indeed.com/jobs?q=Marketing+manager&l=Mexico&start=20
Respuesta de servidor:  <Response [200]>
Número de secciones encontradas: 15
Siguiente url:  https://mx.indeed.com/jobs?q=Marketing+manager&l=Mexico&start=30
Respuesta de servidor:  <Response [200]>
Número de secciones encontradas: 15
Siguiente url:  https://mx.indeed.com/jobs?q=Marketing+manager&l=Mexico&start=40
Respuesta de servidor:  <Response [200]>
Número de secciones encontradas: 15
Siguiente url:  https://mx.indeed.com/jobs?q=Marketing+manager&l=Mexico&start=50
Respuesta de servidor:  <Response [200]>
Número de secciones enco